# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
import logging
import os
import csv
import pickle
import requests
import json

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources
from train import clean_data

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.widgets import RunDetails
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.pipeline.core import Pipeline
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.webservice import Webservice
from azureml.pipeline.steps import AutoMLStep
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

#pip install pyarrow>=0.12.0 --upgrade

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()
ws.write_config(path='.azureml')

# choose a name for experiment
experiment_name = 'Titanic_project'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

amlcompute_cluster_name = "ML-cluster-uda"

try:
      compute_target=ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
      print("Found existing cluster.Using it")
except ComputeTargetException:
      compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
      compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output = True, min_node_count=1, timeout_in_minutes=10)

Workspace name: quick-starts-ws-136684
Azure region: southcentralus
Subscription id: 3d1a56d2-7c81-4118-9790-f85d1acf0c77
Resource group: aml-quickstarts-136684
Found existing cluster.Using it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
found = False
key = "titanic"
description_text = "Titanic survival classification data from Kaggle"

if key in ws.datasets.keys(): 
        found = True
        data = ws.datasets[key] 
else :
    data = pd.read_csv("titanic.csv")

x, y = clean_data(data.to_pandas_dataframe())
x['y']=y
print(x.head(5))
print(x.describe())

# turning pandas dataframe to Tabular data
data_folder = './data'
os.makedirs(data_folder, exist_ok=True)
local_path = 'data/prepared.csv'
x.to_csv(local_path)

datastore = ws.get_default_datastore()
datastore.upload(src_dir='data', target_path='data')

dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, ('data/prepared.csv'))])

   PassengerId  Pclass  Sex Age  SibSp  Parch     Fare  FamilySize  IsAlone  y
0            1       3    0   1      1      0   7.2500           1        0  0
1            2       1    1   2      1      0  71.2833           1        0  1
2            3       3    1   3      0      0   7.9250           0        1  1
3            4       1    1   4      1      0  53.1000           1        0  1
4            5       3    0   5      0      0   8.0500           0        1  0
       PassengerId      Pclass         Sex       SibSp       Parch  \
count   891.000000  891.000000  891.000000  891.000000  891.000000   
mean    446.000000    2.308642    0.352413    0.523008    0.381594   
std     257.353842    0.836071    0.477990    1.102743    0.806057   
min       1.000000    1.000000    0.000000    0.000000    0.000000   
25%     223.500000    2.000000    0.000000    0.000000    0.000000   
50%     446.000000    3.000000    0.000000    0.000000    0.000000   
75%     668.500000    3.000000    1.

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [9]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",   
                             path = "./data",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             n_cross_validations=5,
                             **automl_settings
                            )


In [10]:
# TODO: Submit your experiment
ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

pipeline_run = experiment.submit(pipeline)

Created step automl_module [e8b3974b][be24f2cd-5e5d-4332-b299-69aa3e66e858], (This step will run and generate new outputs)
Submitted PipelineRun b1af25fe-388d-4598-965f-7ef824089048
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/Titanic_project/runs/b1af25fe-388d-4598-965f-7ef824089048?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-136684/workspaces/quick-starts-ws-136684


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [13]:
pipeline_run.wait_for_completion()

PipelineRunId: b1af25fe-388d-4598-965f-7ef824089048
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/Titanic_project/runs/b1af25fe-388d-4598-965f-7ef824089048?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-136684/workspaces/quick-starts-ws-136684
PipelineRun Status: Running


StepRunId: 72ca3d3a-66d0-4ee3-92a8-dfe1534d65da
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/Titanic_project/runs/72ca3d3a-66d0-4ee3-92a8-dfe1534d65da?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-136684/workspaces/quick-starts-ws-136684
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished
{'runId': '72ca3d3a-66d0-4ee3-92a8-dfe1534d65da', 'target': 'ML-cluster-uda', 'status': 'Completed', 'startTimeUtc': '2021-01-31T06:28:32.660278Z', 'endTimeUtc': '2021-01-31T06:56:35.768914Z', 'properties': {'ContentSnapshotId'

'Finished'

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [14]:
# save the best model
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

Downloaded azureml/72ca3d3a-66d0-4ee3-92a8-dfe1534d65da/model_data, 1 files out of an estimated total of 1


Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_samples_split=0.056842105263157895,
                                                                                                    min_weight_fraction_leaf=0.0,
                                                                                                    n_estimators=50,
     

In [15]:
best_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                  feature_sweeping_config=None, feature_sweeping_timeout=None,
                  featurization_config=None, force_text_dnn=None,
                  is_cross_validation=None, is_onnx_compatible=None, logger=None,
                  observer=None, task=None, working_dir=None)),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=None,
                                estimators=[('1',
                                             Pipeline(memory=None,
                                                      steps=[('maxabsscaler',
                                                              MaxAbsScaler(copy=True)),
                                                             ('xgboostclassifier',
                                                              XGBoostClassifier(base_score=0.5,
                                                                  

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [19]:
with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)

best_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_samples_split=0.056842105263157895,
                                                                                                    min_weight_fraction_leaf=0.0,
                                                                                                    n_estimators=50,
     

In [20]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Titanic_survival", description="Titanic Survival pipeline", version="1.0")

published_pipeline

Name,Id,Status,Endpoint
Titanic_survival,4e24159f-ae85-4995-981d-a62f04d8b97e,Active,REST Endpoint


In [21]:
interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

TODO: In the cell below, send a request to the web service you deployed to test it.

In [30]:
rest_endpoint = published_pipeline.endpoint

# Two sets of data to score, so we get two results back
data = {"data":
        [
          {
            "PassengerId": 1709,
            "Pclass": 1,
            "Sex": 0,
            "SibSp": 1,
            "Parch": 0,
            "Fare": 36.05,
            "FamilySize": 1,
            "IsAlone": 0,
            "y": 0
          },
          {
            "PassengerId": 1807,
            "Pclass": 3,
            "Sex":1,
            "SibSp":0,
            "Parch":0,
            "Fare": 72.08,
            "FamilySize": 1,
            "IsAlone":1,
            "y":1
          }
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Make the request and display the response
resp = requests.post(rest_endpoint, headers = auth_header, json = input_data)
print(resp.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

TODO: In the cell below, print the logs of the web service and delete the service

In [28]:
# Set with the deployment name
name = "Titanic_survival"

# load existing web service
service = Webservice(name=name, workspace=ws)

# enable application insight
service.update(enable_app_insights=True)

logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

service.delete()

WebserviceException: WebserviceException:
	Message: WebserviceNotFound: Webservice with name Titanic_survival not found in provided workspace
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "WebserviceNotFound: Webservice with name Titanic_survival not found in provided workspace"
    }
}